In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ../..

D:\GitHub\eeg_analysis


In [2]:
import os
import glob
import json
import dicom2nifti
import nibabel as nib
from tqdm.auto import tqdm
from IPython import display
from pprint import pprint
import pandas as pd

In [ ]:
base_list = glob.glob(r'H:\Other_DB\CAU_EEG\230413_MRI_Integration\*.nii')
base_dict = {os.path.basename(base): (False, None) for base in base_list}


for path in tqdm(glob.glob(r'H:\Other_DB\CAU_EEG\230531_MRI_CAT12_deface\*.nii'), leave=False):
    file_name = os.path.basename(path).strip('anon_')

    proxy = nib.load(path)
    img = proxy.get_fdata()
    base_dict[file_name] = (True, img.shape)

In [ ]:
# pprint(base_dict)

In [ ]:
# Writing to sample.json
with open(r"C:\Users\Minjae\Desktop\cat12_summary.json", "w") as file:
    file.write(json.dumps(base_dict, indent=4))

In [3]:
# mri data subjects
mri_set = set(
    [
        os.path.basename(fname).strip(".nii").split("_")[0]
        for fname in glob.glob(r"H:\Other_DB\CAU_EEG\230413_MRI_Integration\*.nii")
    ]
)

mri_deface_set = set(
    [
        os.path.basename(fname).strip("anon_").strip(".nii").split("_")[0]
        for fname in glob.glob(r"H:\Other_DB\CAU_EEG\230531_MRI_CAT12_deface\*.nii")
    ]
)

# edf data subjects
with open(r"H:\Other_DB\CAU_EEG\201020\annotation_debug.json", 'r') as json_file:
    edf_dict = json.load(json_file)

caueeg_set = set()
for data in edf_dict['data']:
    caueeg_set.add(data['edfname'].split('_')[0])

In [4]:
print(len(caueeg_set))
print(len(mri_set))
print(len(mri_deface_set))

1155
1827
1392


In [5]:
set1 = caueeg_set - mri_set - mri_deface_set
set2 = mri_set - caueeg_set - mri_deface_set
set3 = mri_deface_set - caueeg_set - mri_set

set4 = (caueeg_set & mri_set) - mri_deface_set
set5 = (caueeg_set & mri_deface_set) - mri_set
set6 = (mri_set & mri_deface_set) - caueeg_set

set7 = caueeg_set & mri_set & mri_deface_set

print('set1', len(set1))
print('set2', len(set2))
print('set3', len(set3))
print('set4', len(set4))
print('set5', len(set5))
print('set6', len(set6))
print('set7', len(set7))

set1 555
set2 266
set3 4
set4 175
set5 2
set6 963
set7 423


In [12]:
max_size = max([len(set1), len(set2), len(set3), len(set4), len(set5), len(set6), len(set7)])

table = {}
table['[1]'] = sorted(list(set1)) + ["" for i in range(max_size - len(set1))]
table['[2]'] = sorted(list(set2)) + ["" for i in range(max_size - len(set2))]
table['[3]'] = sorted(list(set3)) + ["" for i in range(max_size - len(set3))]
table['[4]'] = sorted(list(set4)) + ["" for i in range(max_size - len(set4))]
table['[5]'] = sorted(list(set5)) + ["" for i in range(max_size - len(set5))]
table['[6]'] = sorted(list(set6)) + ["" for i in range(max_size - len(set6))]
table['[7]'] = sorted(list(set7)) + ["" for i in range(max_size - len(set7))]

In [13]:
pd.DataFrame(table).to_csv(f'local/output/table.csv')
pd.DataFrame(table).to_excel(f'local/output/table.xlsx')
pd.DataFrame(table)

,[1],[2],[3],[4],[5],[6],[7]
0,00047327,00773096,00725118,00675516,00344923,00000381,00001809
1,00062072,01174160,00743633,01174162,00400036,00000476,00029426
2,00088513,01174818,01003172,01174216,,00000613,00048377
3,00110793,01177737,01008957,01174220,,00000996,00050941
4,00115426,01179889,,01178688,,00001233,00055757
...,...,...,...,...,...,...,...
958,,,,,,01168356,
959,,,,,,01171046,
960,,,,,,01171567,
961,,,,,,01172155,
